In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

retriever = FAISS.load_local("embeddings2/faiss_index", OpenAIEmbeddings(), allow_dangerous_deserialization=True)


/tmp/ipykernel_26170/584869467.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  retriever = FAISS.load_local("embeddings2/faiss_index", OpenAIEmbeddings(), allow_dangerous_deserialization=True)


In [3]:
query = "What was the role of Amatol, NJ during WWI?"
docs = retriever.similarity_search(query, k=10)

for i, doc in enumerate(docs):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content)
    print(f"[Source: {doc.metadata.get('source', 'unknown')}]")



--- Chunk 1 ---
What was the overall impact of Amatol on the surrounding area? There is no reliable and readily-available information to answer this question. Where did the residents of Amatol go once operations ceased? According to the Press of Atlantic City, “Many of Amatol’s laborers, predominantly Irish and Polish from Philadelphia, stayed in the area and started families.”
[Source: amatol_ocr/amatol_sojourn/amatol_sojourn_article.txt]

--- Chunk 2 ---
Loading operations at the Amatol plant began on July 31, 1918, and on August 3, the first shell was loaded. The plant was capable of loading “60,000 shells of all sizes, 50,000 boosters, 50,000 hand grenades, and 20,000 rifle grenades per day, as well as components. To accomplish this work there were 15 smokeless-powder magazines, 33 T.N.T. magazines, 49 miscellaneous storage buildings, 642 plant structures in all.” The plant, during its operational lifespan, loaded over 9,000,000 rounds of ammunition of various types and sizes. 
Al

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
chain = load_qa_with_sources_chain(llm, chain_type="stuff")

response = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
print(response)


{'output_text': 'The role of Amatol, NJ during WWI was primarily as a major military explosives manufacturing and storage site, known as Amatol Arsenal. It was the largest in the country, involved in the production of high explosives such as shells and T.N.T., and supported the war effort by maintaining and operating facilities for the development, production, and storage of explosives, including the loading of smokeless powder into shells. The site employed thousands of workers and was equipped with modern safety measures to handle dangerous materials. It was also planned to be kept in “stand-by” condition for future operations related to explosives production.\n\nSOURCES: amatol_ocr/newspaper_articles_1918/1918-12-14, 2; amatol_ocr/newspaper_articles_1918/1918-02-21, 7; amatol_ocr/newspaper_articles_1919/1919-04-29, 1; amatol_ocr/newspaper_articles_1918/1918-10-08, 6; amatol_ocr/amatol_misc/2016-10-13_amatol_lost_history_summary.txt'}


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

def run_query_with_trace(query, retriever, k=4, model_name="gpt-4.1-nano"):
    print(f"Query: {query}")
    
    # Step 1: Retrieve documents
    docs = retriever.similarity_search(query, k=k)
    
    if not docs:
        print("⚠️ No documents retrieved.")
        return

    print(f"\n🔎 Retrieved {len(docs)} chunks:\n")
    for i, doc in enumerate(docs):
        print(f"--- Chunk {i+1} ---")
        print(doc.page_content.strip())
        print(f"[Source: {doc.metadata.get('source', 'unknown')}]\n")

    # Step 2: Generate answer from chunks
    llm = ChatOpenAI(model=model_name, temperature=0)
    chain = load_qa_with_sources_chain(llm, chain_type="stuff")

    response = chain({
        "input_documents": docs,
        "question": query
    }, return_only_outputs=True)

    print("✅ Final Answer:\n")
    print(response["output_text"])


In [5]:
run_query_with_trace("When did construction begin at Camp Amatol?", retriever, k=10)


Query: When did construction begin at Camp Amatol?

🔎 Retrieved 10 chunks:

--- Chunk 1 ---
Loading operations at the Amatol plant began on July 31, 1918, and on August 3, the first shell was loaded. The plant was capable of loading “60,000 shells of all sizes, 50,000 boosters, 50,000 hand grenades, and 20,000 rifle grenades per day, as well as components. To accomplish this work there were 15 smokeless-powder magazines, 33 T.N.T. magazines, 49 miscellaneous storage buildings, 642 plant structures in all.” The plant, during its operational lifespan, loaded over 9,000,000 rounds of ammunition of various types and sizes. 
Although T.N.T. was preferred and used when available, amatol was used as an alternative due to the shortage of T.N.T. Amatol was “the main charge pressed into the shell (unless straight T.N.T. or a 50/50 mix was used), [and] contains 80 percent of ammonium nitrate and 20 percent of T.N.T.”. The one disadvantage of amatol was that ammonium nitrate absorbs moisture rapid

/tmp/ipykernel_26170/189789882.py:21: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model=model_name, temperature=0)
/tmp/ipykernel_26170/189789882.py:22: LangChainDeprecationWarning: This function is deprecated. Refer to this guide on retrieval and question answering with sources: https://python.langchain.com/docs/how_to/qa_sources/
See also the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank

✅ Final Answer:

Construction at Camp Amatol began in 1918, with loading operations starting on July 31, 1918, and the town reaching completion within nine months of its start, which would place its construction beginning around late 1917 or early 1918. The rapid development was driven by America’s entry into World War I, and the town was completed in just nine months, reaching a population of 7,000. 

SOURCES: amatol_ocr/amatol_sojourn/amatol_sojourn_article.txt


In [6]:
run_query_with_trace("What safety features were implemented at the Amatol munitions factory?", retriever, k=10)


Query: What safety features were implemented at the Amatol munitions factory?

🔎 Retrieved 10 chunks:

--- Chunk 1 ---
Loading operations at the Amatol plant began on July 31, 1918, and on August 3, the first shell was loaded. The plant was capable of loading “60,000 shells of all sizes, 50,000 boosters, 50,000 hand grenades, and 20,000 rifle grenades per day, as well as components. To accomplish this work there were 15 smokeless-powder magazines, 33 T.N.T. magazines, 49 miscellaneous storage buildings, 642 plant structures in all.” The plant, during its operational lifespan, loaded over 9,000,000 rounds of ammunition of various types and sizes. 
Although T.N.T. was preferred and used when available, amatol was used as an alternative due to the shortage of T.N.T. Amatol was “the main charge pressed into the shell (unless straight T.N.T. or a 50/50 mix was used), [and] contains 80 percent of ammonium nitrate and 20 percent of T.N.T.”. The one disadvantage of amatol was that ammonium nit

In [7]:
run_query_with_trace("What was the opinion of Col. Hawkins regarding the town and plant of Amatol, NJ?", retriever, k=10)


Query: What was the opinion of Col. Hawkins regarding the town and plant of Amatol, NJ?

🔎 Retrieved 10 chunks:

--- Chunk 1 ---
Loading operations at the Amatol plant began on July 31, 1918, and on August 3, the first shell was loaded. The plant was capable of loading “60,000 shells of all sizes, 50,000 boosters, 50,000 hand grenades, and 20,000 rifle grenades per day, as well as components. To accomplish this work there were 15 smokeless-powder magazines, 33 T.N.T. magazines, 49 miscellaneous storage buildings, 642 plant structures in all.” The plant, during its operational lifespan, loaded over 9,000,000 rounds of ammunition of various types and sizes. 
Although T.N.T. was preferred and used when available, amatol was used as an alternative due to the shortage of T.N.T. Amatol was “the main charge pressed into the shell (unless straight T.N.T. or a 50/50 mix was used), [and] contains 80 percent of ammonium nitrate and 20 percent of T.N.T.”. The one disadvantage of amatol was that am

In [25]:
run_query_with_trace("What did Colonel Hawkins say about amenities for Amatol, NJ?", retriever, k=10)


Query: What did Colonel Hawkins say about amenities for Amatol, NJ?

🔎 Retrieved 10 chunks:

--- Chunk 1 ---
Stenographers, Clerks, etc.: 2

The Company had about 5,500 men on construction work, including all trades such as:

Carpenters

Plumbers

Electricians

Plasterers

Lathe workers

Machinists

Boiler makers

Painters

Teamsters

Laborers
[Source: amatol_ocr/amatol_book/p-25.txt]

--- Chunk 2 ---
Electrical Engineer: 1

Material Engineer: 1

Development Engineer: 1

Civil and Structural Engineer: 1

Consulting Architect: 1

Architect: 1

Cost Engineer: 1

Chief Chemist: 1

Assistant Chemists: 3

Engineers and Draftsmen: 7

Field Men (Instruments, Rodmen, etc.): 65

Clerks, Stenographers, etc.: 9

CONSTRUCTION
General Construction Superintendent: 1

Assistant General Construction Superintendent: 1

Construction Engineers: 10

Foremen and Sub-Foremen: 54

Stenographers, Clerks, etc.: 2
[Source: amatol_ocr/amatol_book/p-25.txt]

--- Chunk 3 ---
DATA ON HIGH EXPLOSIVE SHELL
Size	Weigh

In [26]:
run_query_with_trace("Why was Camp Dix not selected as a site for the munitions factory?", retriever, k=10)


Query: Why was Camp Dix not selected as a site for the munitions factory?

🔎 Retrieved 10 chunks:

--- Chunk 1 ---
Stenographers, Clerks, etc.: 2

The Company had about 5,500 men on construction work, including all trades such as:

Carpenters

Plumbers

Electricians

Plasterers

Lathe workers

Machinists

Boiler makers

Painters

Teamsters

Laborers
[Source: amatol_ocr/amatol_book/p-25.txt]

--- Chunk 2 ---
The Allied Building Trades’ Protective As-  
sociation perfected permanent organization at  
a meeting last night. Its first official action  
was the passing of a resolution disapproving of  
the demand of striking mechanics for an in-  
crease in wages from $5 to $7 a day. W. E.  
Soper was elected president at this meeting.  
Other officers are: Vice-president, Samuel  
Guss; secretary, W. H. Hannum; treasurer,  
Mr. Philip Hannum.
[Source: amatol_ocr/newspaper_articles_1919/1919-01-12, 9, Philadelphia Inquirer, work at plants halt.txt]

--- Chunk 3 ---
Engineers of the War Depar

In [28]:
run_query_with_trace("What did Lieutenant Colonel R.H. Hawkins say about why Camp Dix was not selected as a site for the munitions factory?", retriever, k=10)


Query: What did Lieutenant Colonel R.H. Hawkins say about why Camp Dix was not selected as a site for the munitions factory?

🔎 Retrieved 10 chunks:

--- Chunk 1 ---
Stenographers, Clerks, etc.: 2

The Company had about 5,500 men on construction work, including all trades such as:

Carpenters

Plumbers

Electricians

Plasterers

Lathe workers

Machinists

Boiler makers

Painters

Teamsters

Laborers
[Source: amatol_ocr/amatol_book/p-25.txt]

--- Chunk 2 ---
The Allied Building Trades’ Protective As-  
sociation perfected permanent organization at  
a meeting last night. Its first official action  
was the passing of a resolution disapproving of  
the demand of striking mechanics for an in-  
crease in wages from $5 to $7 a day. W. E.  
Soper was elected president at this meeting.  
Other officers are: Vice-president, Samuel  
Guss; secretary, W. H. Hannum; treasurer,  
Mr. Philip Hannum.
[Source: amatol_ocr/newspaper_articles_1919/1919-01-12, 9, Philadelphia Inquirer, work at plants ha